# 내삽법<br>Interpolation

예를 들어 실험을 통해 얻은 두 측정값 사이의 값을 추정하고 싶을 경우, 내삽법을 시도해 볼 수 있다.<br>For instance, to guess values between two measurements, we may try interpolation.

아래의 표를 살펴보자.<br>Let's take a look at the following table.

In [ ]:
# Import NumPy and matplotlib
import pylab as py



In [ ]:
# Import pandas for tables
import pandas as pd

# What is this?
py.seed()

# Parameters
a = 0.5
b = 1.5

# x array
x_array = py.arange(5+0.5)

# True values of y
y_true = a * x_array + b

# contamination
noise = py.random(x_array.shape) - 0.5

# Measurement values
y_measurement = y_true + noise

# Organize data in table form
# https://stackoverflow.com/questions35160256
df = pd.DataFrame(
    {'$x$':x_array, '$y_{true}$':y_true, '$y_{measurement}$':y_measurement}, 
    columns=['$x$', '$y_{true}$', '$y_{measurement}$'],
)

# Plot data points
ax = df.plot.line(x='$x$', y='$y_{true}$')
df.plot.scatter(x='$x$', y='$y_{measurement}$', ax=ax, label='$y_{measurement}$')

py.show()



In [ ]:
# Present the table
df



여기서 $0 \le x \le 1$ 구간의 $y$ 값을 알아보자.<br>
Let's try to figure out $y$ values in the $0 \le x \le 1$ interval.

## 선형 내삽<br>Linear interpolation

두 점 $(x_1, y_1)$, $(x_2, y_2)$ 을 지나는 직선의 방정식을 구할 수 있다.<br>We can formulate the straight line passing two points of $(x_1, y_1)$ and $(x_2, y_2)$.

In [ ]:
# Import symbolic processor module
import sympy as sy

# Initialize printing equations
sy.init_printing()



In [ ]:
# Declare symbols
x = sy.symbols('x')

# Multiple symbols using `:`
x1, x2 = sy.symbols('x1:3')
y1, y2 = sy.symbols('y1:3')

# Define slope
slope = (y2 - y1) / (x2 - x1)

# Define the straight line
y_interp = slope * (x - x1) + y1

# Present the equation
y_interp



($0 \le x \le 1$ 구간에서) 임의의 $x_i$ 에 대응되는 $y_i$ 는 다음과 같이 구할 수 있다.<br>We can find $y_i$ for an arbitrary $x_i$ (within $0 \le x \le 1$ interval) as follows.

In [ ]:
# Declared x_i as a SymPy symbol
x_i = sy.symbols('x_i')

# Prepared a dictionary containing substitution pairs
substitution_dict = {
    # "substitute x with x_i"
    x: x_i,
    x1: x_array[0],
    x2: x_array[1],
    y1: y_measurement[0],
    y2: y_measurement[1],
}

# Substitution
y_i_sy = y_interp.subs(substitution_dict)

# Result of substitution
y_i_sy



프로그래밍 언어 구문을 생성하는 것도 가능하다.<br>SymPy may generate expressions in programming languages.

In [ ]:
python_code = sy.python(y_interp)
print(python_code)



In [ ]:
c_code = sy.ccode(y_interp)
print(c_code)



In [ ]:
fortran_code = sy.fcode(y_interp)
print(fortran_code)



보통 `interp()` 함수를 이용한다.<br>Usually we call `interp()` function.

In [ ]:
# x values to interpolate
x_i = py.linspace(x_array[0], x_array[-1], 50+1)

# Interpolate
y_i = py.interp(x_i, x_array, y_measurement)



In [ ]:
# Plot data points
ax = df.plot.line(x='$x$', y='$y_{true}$')
df.plot.scatter(x='$x$', y='$y_{measurement}$', ax=ax, label='$y_{measurement}$')

# Plot interpolation
ax.plot(x_i, y_i, '.', label='$y_{interp}$')

# Show legend table
py.legend(loc=0)

py.show()



### `pandas`

판다스의 데이터프레임도 간단한 내삽 기능이 있다.<br>`DataFrame` of `pandas` also has simple interpolation features.

In [ ]:
df_interp_nan = df.reindex(x_i)



In [ ]:
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.interpolate.html
df_interp = df_interp_nan.interpolate(method='linear')



In [ ]:
# Plot data points
ax = df.plot.line(x='$x$', y='$y_{true}$')
# Plot interpolation
df_interp.plot.scatter(x='$x$', y='$y_{measurement}$', ax=ax, label='$y_{interp}$', c='orange')
df.plot.scatter(x='$x$', y='$y_{measurement}$', ax=ax, label='$y_{measurement}$')

# Show legend table
py.legend(loc=0)

py.show()



다음은 $y_{measurement}$ 의 히스토그램을 그린 것이다.<br>Following shows a histogram of $y_{measurement}$.

In [ ]:
df_interp['$y_{measurement}$'].hist()



## 3차 스플라인 곡선<br>Cubic spline curve

[스플라인](https://en.wiktionary.org/wiki/spline)은 얇고 긴 나무자를 말한다. 부드러운 곡선을 그리기 위해 사용했었다.<br>A [spline](https://en.wiktionary.org/wiki/spline) is a ruler made of a piece of thin and long rectangular wood.  Drafters used it draw a smooth curve.

여기서 "3차"는 내삽할 때 3차 다항식 ($a_0 x^3 + a_1 x^2 + a_2 x + a_3$) 을 사용한다는 의미이다.<br>"Cubic" here means that we would interpolate using a 3rd order polynomial ($a_0 x^3 + a_1 x^2 + a_2 x + a_3$).

### SciPy

In [ ]:
# https://www.scipy-lectures.org/intro/scipy.html#interpolation-scipy-interpolate

# Import interpolation subpackage
import scipy.interpolate as sn

cubic_interp = sn.interp1d(x_array, y_measurement, kind='cubic')
y_cubic = cubic_interp(x_i)



In [ ]:
# Plot data points
ax = df.plot.line(x='$x$', y='$y_{true}$')
df.plot.scatter(x='$x$', y='$y_{measurement}$', ax=ax, label='$y_{measurement}$')

# Plot linear interpolation
ax.plot(x_i, y_i, '.', label='$y_{linear}$')

# Plot cubic spline curve
ax.plot(x_i, y_cubic, 'x', label='$y_{cubic}$')

# Show legend table
py.legend(loc=0)

py.show()



### pandas

In [ ]:
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.interpolate.html
df_interp = df_interp_nan.interpolate(method='cubic')



In [ ]:
# Plot data points
ax = df.plot.line(x='$x$', y='$y_{true}$')
# Plot interpolation
df_interp.plot.scatter(x='$x$', y='$y_{measurement}$', ax=ax, label='$y_{interp}$', c='orange')
df.plot.scatter(x='$x$', y='$y_{measurement}$', ax=ax, label='$y_{measurement}$')

# Show legend table
py.legend(loc=0)

py.show()



도전 과제 1 : $\theta_0=45(^\circ)$, $\theta_1=90(^\circ)$, $\theta_2=135(^\circ)$ 일 때 $\theta_0=45(^\circ)$ ~ $\theta_2=135(^\circ)$ 구간에서 $sin \theta^\circ $ 그래프를 그려 보시오.<br>Try this 1 : Plot the $sin \theta^\circ$ within $\theta_0=45(^\circ)$ ~ $\theta_2=135(^\circ)$ interval. Here, $\theta_0=45(^\circ)$, $\theta_1=90(^\circ)$, and $\theta_2=135(^\circ)$.



도전 과제 2 : $y = a_0 \theta^2 + a_1 \theta + a_2$가 위 세 점에서 $sin\theta^\circ$ 와 같은 값을 가지게 되는 $a_0$, $a_1$, $a_2$ 값을 구해보시오.<br>Try this 2 : Find $a_0$, $a_1$, and $a_2$ values that make $y = a_0 \theta^2 + a_1 \theta + a_2$ have the same values as $sin \theta^\circ$ at above three points.



* hint

In [ ]:
import sympy as sy

a, b = sy.symbols('a, b')
x = sy.symbols('x1:3')
y = sy.symbols('y1:3')

a, b, x, y



In [ ]:
eq1 = sy.Eq(a*x1 + b, y1)
eq2 = sy.Eq(a*x2 + b, y2)

sol = sy.solve((eq1, eq2), (a, b))
sol



도전 과제 3 : 위에서 구한 $a_0, a_1, a_2$ 값으로 $\theta_0$ ~ $\theta_2$ 구간의 $y$ 와 $sin\theta^\circ$ 그래프를 1도 단위로 그려보고 비교하시오.<br>Try this 3 : Using $a_0$, $a_1$, and $a_2$ values above, plot $y$ and $sin\theta^\circ$ within $\theta_0$ ~ $\theta_2$ interval with $1^\circ$ pitch and compare.

